# Análisis Exploratorio de Datos (EDA) de Velocidad de Internet por Provincia en Argentina
**Autor:** Matías Oviedo

## Introducción  
En este notebook, se realizará un Análisis Exploratorio de Datos (EDA) utilizando un conjunto de datos que reflejan las conexiones a internet y telefonicas existentes de Argentina en el primer trimestre de 2024. El objetivo es analizar los diferentes datos, y definir lineas de tendencia para saber el futuro sobre tecnologías y planes de internet. El analisis se hará con la librería plotly, la cual es excelente para crear graficos interactivos.

El anális está creado en base a un proyecto de investsigación de [Soy Henry](https://github.com/soyHenry/PI_DA/tree/PART-TIME) y los datasets se obtienen de [Enacom](https://indicadores.enacom.gob.ar/datos-abiertos)


## Datasets:
Los archivos se descargan en formato .xlsx, pero en la limpieza de datos con el ETL se convierten a .csv para facilitar el manjeo de los mismos.


## EDA:

### Importaciones de librerías
Comenzamos importanto todas las liberías que utilizaremos en el codigo y definiendo algunas variables claves

In [27]:
import pandas as pd
import seaborn as sns
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import geopandas as gpd
from sklearn.linear_model import LinearRegression
from IPython.display import display,HTML
from datetime import datetime
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

cf.set_config_file(sharing='public',theme='ggplot',offline=True) 

carpeta = 'datasets'
ano_actual = datetime.now().year
sns.set(style="whitegrid") #estilo de grafico

### Analisis 1:
* Objetivo: Identificar la media de mbps de decarga por provincias para entender el desarrollo de las mismas.
* Metodo: Crear un grafico interactivo con Plotly coloreando provincias de un GEOJSON

In [28]:
#leer el archivo csv
df = pd.read_csv('datasets/Internet_Velocidad_%_por_prov.csv')

#filtramos datos del ano actual
df_2024 = df[df['año'] == ano_actual]

#importar el archivo geojson
geojson_path = 'datasets/mapas/map.geojson'
gdf = gpd.read_file(geojson_path)

#reorganizar valores nulos y df
gdf['mbps_(media_de_bajada)'] = gdf['nombre'].map(df_2024.set_index('provincia')['mbps_(media_de_bajada)'])
gdf['mbps_(media_de_bajada)'] = gdf['mbps_(media_de_bajada)'].fillna(0)  #asignar 0 donde no hay datos

#creamos el grafico
fig = go.Figure()

#anador geojson como un mapa
fig.add_trace(go.Choroplethmapbox(
    geojson=gdf.__geo_interface__,
    locations=gdf.index,
    z=gdf['mbps_(media_de_bajada)'],
    colorscale='Viridis',
    zmin=0,
    zmax=gdf['mbps_(media_de_bajada)'].max(),
    marker_opacity=0.7,
    marker_line_width=0.5,
    marker_line_color='black',
    colorbar=dict(title='Mbps (media de bajada)'),
    hoverinfo='text',
    text=gdf['nombre'] + ': ' + gdf['mbps_(media_de_bajada)'].astype(str) + ' Mbps',
))

#configurar posicion de mapa
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox_zoom=2.8,
    mapbox_center={"lat": -38.4161, "lon": -63.6167},  #centro de argentina
    title_text=f'Velocidad Media de Internet por Provincia en Argentina ({ano_actual})',
)

#mostrar grafico
fig.show()


### Analisis 2:
* Objetivo: Identificar las tendencias de las tecnologías para identificar su futuro
* Metodo: Crear un grafico de lineas marcando los valores historicos y graficando linea de tendencia

In [29]:
#leer csv
df_tecnologia_agrupado = pd.read_csv('datasets/Internet_Accesos_Por_Tecnologia.csv')

#limpiar y convertir a numeros
df_tecnologia_agrupado['año'] = pd.to_numeric(df_tecnologia_agrupado['año'], errors='coerce')
df_tecnologia_agrupado['adsl'] = pd.to_numeric(df_tecnologia_agrupado['adsl'], errors='coerce')
df_tecnologia_agrupado['cablemodem'] = pd.to_numeric(df_tecnologia_agrupado['cablemodem'], errors='coerce')
df_tecnologia_agrupado['fibra_óptica'] = pd.to_numeric(df_tecnologia_agrupado['fibra_óptica'], errors='coerce')
df_tecnologia_agrupado['wireless'] = pd.to_numeric(df_tecnologia_agrupado['wireless'], errors='coerce')
df_tecnologia_agrupado['otros'] = pd.to_numeric(df_tecnologia_agrupado['otros'], errors='coerce')

#eliminar filas con nulos
df_tecnologia_agrupado = df_tecnologia_agrupado.dropna()

for index in df_tecnologia_agrupado[df_tecnologia_agrupado['año'] == 2024].index:
    df_tecnologia_agrupado.drop(index, inplace=True)

#nombres de tecnologias
tecnologias = ['adsl', 'cablemodem', 'fibra_óptica', 'wireless', 'otros']

#colores para tecnologias
colores = ['blue', 'orange', 'green', 'red', 'purple']

#agrupar datos por ano y tecnologia
df_agrupado = df_tecnologia_agrupado.groupby('año')[tecnologias].sum().reset_index()

#crear grafico inicial
fig = px.scatter(title='Tendencia tecnologías utilizadas para internet en Argentina',
                 labels={'año': 'Año', 'value': 'Accesos'})



#agregar datos al grafico
for i, tecnologia in enumerate(tecnologias):
    #graficar los puntos de datos (un solo punto por ano)
    scatter_fig = px.scatter(df_agrupado, x='año', y=tecnologia, labels={'año': 'Año', 'value': tecnologia})
    fig.add_trace(scatter_fig.data[0].update(marker_color=colores[i], name=tecnologia))
    
    #crear y ajustar el modelo de regresion lineal
    X = df_agrupado[['año']]
    y = df_agrupado[tecnologia]

    model = LinearRegression()
    model.fit(X, y)

    #predecir para los anos presentes y futuros
    future_years = np.array(list(df_agrupado['año']) + [2024, 2025, 2026]).reshape(-1, 1)
    predictions = model.predict(future_years)

    #graficar linea de tendencia
    line_fig = px.line(x=future_years.flatten(), y=predictions, labels={'x': 'Año', 'y': tecnologia})
    fig.add_trace(line_fig.data[0].update(line_color=colores[i], name=f'Tendencia {tecnologia}'))

#ajustar el diseno
fig.update_layout(legend_title_text='Tecnología')

#mostrar grafico
fig.show()


c:\Users\Mati\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\Mati\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\Mati\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\Mati\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\Mati\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names



### Analisis 3:
* Objetivo: Identificar las tendencias de las velocidades de internet para identificar su futuro
* Metodo: Crear un grafico de lineas marcando los valores historicos y graficando linea de tendencia

In [30]:
#cargar los datos
df_tec_vel_agrupado = pd.read_csv('datasets/Internet_Accesos_Por_velocidad.csv')

#limpiar y convertir a numeros
df_tec_vel_agrupado['año'] = pd.to_numeric(df_tec_vel_agrupado['año'], errors='coerce')
df_tec_vel_agrupado['hasta_512_kbps'] = pd.to_numeric(df_tec_vel_agrupado['hasta_512_kbps'], errors='coerce')
df_tec_vel_agrupado['+_512_kbps_-_1_mbps'] = pd.to_numeric(df_tec_vel_agrupado['+_512_kbps_-_1_mbps'], errors='coerce')
df_tec_vel_agrupado['+_1_mbps_-_6_mbps'] = pd.to_numeric(df_tec_vel_agrupado['+_1_mbps_-_6_mbps'], errors='coerce')
df_tec_vel_agrupado['+_6_mbps_-_10_mbps'] = pd.to_numeric(df_tec_vel_agrupado['+_6_mbps_-_10_mbps'], errors='coerce')
df_tec_vel_agrupado['+_10_mbps_-_20_mbps'] = pd.to_numeric(df_tec_vel_agrupado['+_10_mbps_-_20_mbps'], errors='coerce')
df_tec_vel_agrupado['+_20_mbps_-_30_mbps'] = pd.to_numeric(df_tec_vel_agrupado['+_20_mbps_-_30_mbps'], errors='coerce')
df_tec_vel_agrupado['+_30_mbps'] = pd.to_numeric(df_tec_vel_agrupado['+_30_mbps'], errors='coerce')
df_tec_vel_agrupado['otros'] = pd.to_numeric(df_tec_vel_agrupado['otros'], errors='coerce')
#borrar filas con nulos
df_tec_vel_agrupado = df_tec_vel_agrupado.dropna()

for index in df_tec_vel_agrupado[df_tec_vel_agrupado['año'] == 2024].index:
    df_tec_vel_agrupado.drop(index, inplace=True)

tecnologias = ['hasta_512_kbps','+_512_kbps_-_1_mbps','+_1_mbps_-_6_mbps','+_6_mbps_-_10_mbps','+_10_mbps_-_20_mbps','+_20_mbps_-_30_mbps','+_30_mbps','otros']

colores = ['blue', 'orange', 'green', 'red', 'purple','yellow','black','pink']

#agrupar por ano y tecnologia
df_agrupado = df_tec_vel_agrupado.groupby('año')[tecnologias].sum().reset_index()

#crear grafico incial
fig = px.scatter(title='Tendencia de velocidades de internet en Argentina',
                 labels={'año': 'Año', 'value': 'Accesos'})

#agregar datos
for i, tecnologia in enumerate(tecnologias):
    #graficar los puntos de datos (un solo punto por ano)
    scatter_fig = px.scatter(df_agrupado, x='año', y=tecnologia, labels={'año': 'Año', 'value': tecnologia})
    fig.add_trace(scatter_fig.data[0].update(marker_color=colores[i], name=tecnologia))
    
    #crear y ajustar el modelo de regresion lineal
    X = df_agrupado[['año']]
    y = df_agrupado[tecnologia]

    model = LinearRegression()
    model.fit(X, y)

    #predecir para los anos presentes y futuros
    future_years = np.array(list(df_agrupado['año']) + [2024, 2025, 2026]).reshape(-1, 1)
    predictions = model.predict(future_years)

    #graficar linea de tendencia
    line_fig = px.line(x=future_years.flatten(), y=predictions, labels={'x': 'Año', 'y': tecnologia})
    fig.add_trace(line_fig.data[0].update(line_color=colores[i], name=f'Tendencia {tecnologia}'))

#ajustar al diseno
fig.update_layout(legend_title_text='Tecnología')

#mostrar grafico
fig.show()


c:\Users\Mati\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\Mati\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\Mati\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\Mati\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\Mati\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\Mati\AppDat

### Analisis 4:
* Objetivo: Identificar las provincias con más accesos a internet cada 100 hogares, también crecimiento y tendencias
* Metodo: Crear un grafico de lineas marcando los valores historicos y graficando linea de tendencia

In [31]:
#leer csv
df_acc_hog = pd.read_csv('datasets/Internet_Penetracion-hogares.csv')

#asegurarse de que los anos sean numericos
df_acc_hog['año'] = pd.to_numeric(df_acc_hog['año'], errors='coerce')

#borrar filas con nulos
df_acc_hog = df_acc_hog.dropna()

#agrupar por ano y provincia, calculando la media de accesos_por_cada_100_hogares
df_grouped = df_acc_hog.groupby(['año', 'provincia'])['accesos_por_cada_100_hogares'].mean().reset_index()

#crear grafico inicial
fig = go.Figure()

#listar provincias
provincias = df_grouped['provincia'].unique()

#definir colores
colores = px.colors.qualitative.Set1

#graficar la tendencia por provincia como antes
for i, provincia in enumerate(provincias):
    #filtrar los datos por provincia
    df_provincia = df_grouped[df_grouped['provincia'] == provincia]
    
    #graficar los puntos originales con la media por ano
    fig.add_trace(go.Scatter(x=df_provincia['año'], 
                             y=df_provincia['accesos_por_cada_100_hogares'],
                             mode='lines+markers',
                             name=provincia,
                             line=dict(color=colores[i % len(colores)], width=2)))

    #crear y ajustar un modelo de regresion lineal
    X = df_provincia[['año']]
    y = df_provincia['accesos_por_cada_100_hogares']
    
    model = LinearRegression()
    model.fit(X, y)

    #predecir los valores hasta 2026
    future_years = np.array(range(df_grouped['año'].min(), 2027)).reshape(-1, 1)
    predictions = model.predict(future_years)

    #graficar la linea de tendencia suavizada
    fig.add_trace(go.Scatter(x=future_years.flatten(), 
                             y=predictions,
                             mode='lines',
                             name=f'Tendencia {provincia}',
                             line=dict(dash='dash', color=colores[i % len(colores)], width=2)))

#calculo del kpi del incremento del 2%
ultimo_año = df_grouped['año'].max()
df_ultimo_año = df_grouped[df_grouped['año'] == ultimo_año]

#anadir una columna con el objetivo del incremento del 2%
df_ultimo_año['objetivo_2'] = df_ultimo_año['accesos_por_cada_100_hogares'] * 1.02

#graficar los puntos del objetivo del incremento del 2%
for i, provincia in enumerate(provincias):
    df_provincia_objetivo = df_ultimo_año[df_ultimo_año['provincia'] == provincia]
    fig.add_trace(go.Scatter(x=[ultimo_año + 1], 
                             y=df_provincia_objetivo['objetivo_2'],
                             mode='markers',
                             name=f'Objetivo 2% {provincia}',
                             marker=dict(symbol='x', color=colores[i % len(colores)], size=10)))

#ajustar el diseno del grafico
fig.update_layout(
    title='Accesos por Cada 100 Hogares - Crecimiento, Tendencia y Objetivo del KPI por Provincia',
    xaxis_title='Año',
    yaxis_title='Accesos por Cada 100 Hogares',
    legend_title='Provincia',
    hovermode='x unified',
    template='plotly_white'
)

#mostrar el grafico
fig.show()


c:\Users\Mati\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\Mati\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\Mati\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\Mati\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\Mati\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\Mati\AppDat

### Analisis 5:
* Objetivo: Identificar las provincias con más accesos a internet cada 100 habitantes, también crecimiento y tendencias
* Metodo: Crear un grafico de lineas marcando los valores historicos y graficando linea de tendencia

In [32]:
#leer csv
df_pen_pob = pd.read_csv('datasets/Internet_Penetracion-poblacion.csv')

#asegurar que los anos sean numeros
df_pen_pob['año'] = pd.to_numeric(df_pen_pob['año'], errors='coerce')

#eliminar filas con nulos
df_pen_pob = df_pen_pob.dropna()

#agrupar por ano y provincia, calculando la media de accesos_por_cada_100_hogares
df_grouped = df_pen_pob.groupby(['año', 'provincia'])['accesos_por_cada_100_hab'].mean().reset_index()

#crear grafico inicial
fig = go.Figure()

#listar provincias
provincias = df_grouped['provincia'].unique()

#colores para cada provincia
colores = px.colors.qualitative.Set1

#agregar una linea para cada provincia, marcando solo un punto por ano (promedio)
for i, provincia in enumerate(provincias):
    #filtrar los datos por provincia
    df_provincia = df_grouped[df_grouped['provincia'] == provincia]
    
    #graficar los puntos originales con la media por ano
    fig.add_trace(go.Scatter(x=df_provincia['año'], 
                             y=df_provincia['accesos_por_cada_100_hab'],
                             mode='lines+markers',
                             name=provincia,
                             line=dict(color=colores[i % len(colores)], width=2)))

    #ajustar modelo
    X = df_provincia[['año']]
    y = df_provincia['accesos_por_cada_100_hab']
    
    model = LinearRegression()
    model.fit(X, y)

    #predecir los valores hasta 2026
    future_years = np.array(range(df_grouped['año'].min(), 2027)).reshape(-1, 1)
    predictions = model.predict(future_years)

    #graficar linea suavizada
    fig.add_trace(go.Scatter(x=future_years.flatten(), 
                             y=predictions,
                             mode='lines',
                             name=f'Tendencia {provincia}',
                             line=dict(dash='dash', color=colores[i % len(colores)], width=2)))

#ajustar disneo
fig.update_layout(
    title='Accesos por Cada 100 habitantes - Tendencia por Provincia',
    xaxis_title='Año',
    yaxis_title='Accesos por Cada 100 Hogares',
    legend_title='Provincia',
    hovermode='x unified',
    template='plotly_white'
)

#mostrar grafico
fig.show()


c:\Users\Mati\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\Mati\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\Mati\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\Mati\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\Mati\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names

c:\Users\Mati\AppDat

### Analisis 6:
* Objetivo: conocer el desarrollo en conexiones a internet conociendo la media de mbps
* Metodo: Crear un grafico de lineas marcando los valores historicos

In [33]:
#leer csv
df_media_mbps = pd.read_csv('datasets/Internet_Totales_VMD.csv')

#omitir valores del ano actual
df_media_mbps = df_media_mbps[df_media_mbps['año'] != ano_actual]

#agrupar y calcular media
df_annual_mbps = df_media_mbps.groupby('año').agg({'mbps_(media_de_bajada)': 'mean'}).reset_index()

#transformar el objetivo para un ajuste exponencial
df_annual_mbps['log_mbps'] = np.log(df_annual_mbps['mbps_(media_de_bajada)'])

#ajustar un modelo de regresion lineal a la transformacion logaritmica (ajuste exponencial)
X = df_annual_mbps['año'].values.reshape(-1, 1)
y_log = df_annual_mbps['log_mbps'].values
model = LinearRegression()
model.fit(X, y_log)

#predecir el valor logaritmico para 2024 y convertirlo de nuevo a su escala original
predicted_log_mbps_2024 = model.predict([[ano_actual]])
predicted_mbps_2024 = np.exp(predicted_log_mbps_2024)

#agregar la prediccion de 2024 al dataframe
df_predicted_2024 = pd.DataFrame({'año': [ano_actual], 'mbps_(media_de_bajada)': predicted_mbps_2024})
df_combined = pd.concat([df_annual_mbps[['año', 'mbps_(media_de_bajada)']], df_predicted_2024])

#crear grafico inicial
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_combined['año'], 
    y=df_combined['mbps_(media_de_bajada)'], 
    mode='lines+markers', 
    name='Media de Mbps',
    line=dict(color='blue'),
    marker=dict(size=8)
))

#personalizar
fig.update_layout(
    title='Evolución de la Velocidad Media de internet en Argentina',
    xaxis_title='Año',
    yaxis_title='Mbps (Media de Bajada)',
    hovermode='x',
    template='plotly_white'
)

#mostrar grafico y resultados
print(f'El valor predicho para {ano_actual} es {predicted_mbps_2024[0]:.2f} Mbps.')
fig.show()



El valor predicho para 2024 es 175.68 Mbps.


### Analisis 7:
* Objetivo: Identificar el tamaño de la Proporción del Total de Mbps por Provincia
* Metodo: Crear un grafico circular que represente la suma de los Mbps vendidos

In [34]:
#cargar los datos del csv
df_vel_s_ran = pd.read_csv('datasets/Internet_Velocidad_sin_Rangos.csv')

#eliminar columnas irrelevantes si es necesario (como "otros" u otras)
df_vel_s_ran.drop(columns=["otros"], inplace=True)

#seleccionar las columnas de velocidades (ignoramos las que no tienen formato numerico)
vel_columns = ["0,256_mbps", "0,5_mbps", "0,512_mbps", "0,75_mbps", "1_mbps", "2_mbps", "3_mbps", 
               "3,5_mbps", "4_mbps", "5_mbps", "6_mbps", "7_mbps", "8_mbps", "9_mbps", "10_mbps", 
               "11_mbps", "12_mbps", "15_mbps", "16_mbps", "18_mbps", "20_mbps", "24_mbps", "25_mbps", 
               "30_mbps", "35_mbps", "40_mbps", "50_mbps", "60_mbps", "100_mbps", "120_mbps", 
               "150_mbps", "200_mbps", "300_mbps", "500_mbps", "1000_mbps"]

#convertir las columnas a numericas, eliminando las comas
df_vel_s_ran[vel_columns] = df_vel_s_ran[vel_columns].replace(',', '.', regex=True).apply(pd.to_numeric, errors='coerce')

#calcular el total de mbps por provincia sumando todas las columnas de velocidad
df_vel_s_ran['total_mbps'] = df_vel_s_ran[vel_columns].sum(axis=1)

#ordenar por el total de mbps para el ranking
df_ranking = df_vel_s_ran[['provincia', 'total_mbps']].sort_values(by='total_mbps', ascending=False)

fig = px.pie(df_ranking, 
             values='total_mbps', 
             names='provincia', 
             title='Proporción del Total de Mbps por Provincia')

#personalizar el grafico
fig.update_traces(textposition='inside', textinfo='percent+label')

#mostrar el grafico
fig.show()


### Analisis 8:
* Objetivo: Identificar el tamaño de las empresas de portabilidad, e identificar su tendencia
* Metodo: Crear un grafico de lineas que muestre graficos historicos

In [35]:
#leer csv
df_portabilidad = pd.read_csv('datasets/Portabilidad_Portin.csv')

#borrar columnas que no se usaran
df_portabilidad.drop(columns=['total_general'], inplace=True)

#reemplazar valores negativos por su valor absoluto
df_portabilidad.iloc[:, 2:] = df_portabilidad.iloc[:, 2:].abs()

#reorganizar el dataframe para el grafico
df_melted = df_portabilidad.melt(id_vars=['año', 'mes'], var_name='empresa', value_name='abonos')

#calcular el promedio de abonados por ano y empresa
df_annual = df_melted.groupby(['año', 'empresa'])['abonos'].mean().reset_index()

#crear el grafico de lineas con promedios anuales
fig = px.line(df_annual, 
              x='año', 
              y='abonos', 
              color='empresa', 
              title='Promedio Anual de Abonados por Empresa de Telecomunicaciones',
              labels={'abonos': 'Cantidad de Abonos', 'año': 'Año'},
              markers=True)

#mostrar el grafico
fig.show()


### Analisis 9:
* Objetivo: Identificar el tamaño actual de las empresas de portabilidad
* Metodo: Crear un grafico circular representando el tamaño de las empresas en cantidad de abonos totales

In [36]:
#leer csv
df_portabilidad = pd.read_csv('datasets/Portabilidad_Portin.csv')
df_portabilidad.drop(columns=['nextel'], inplace=True)

#elegir el ultimo valor
last_row = df_portabilidad.iloc[0]

#organizar df
df_size = last_row[2:-1].reset_index()  #ignorar 'ano', 'mes' y 'periodo'
df_size.columns = ['empresa', 'abonos']  #renombrar columnas

#definir colores de empresa
color_map = {
    'personal': 'blue',
    'claro': 'orange',
    'movistar': 'red'
}

#crear grafico inicial
fig = px.pie(df_size, 
             names='empresa', 
             values='abonos', 
             title='Tamaño de Abonados por Empresa (Última Medición)',
             hole=0.3,  #hueco cona
             color_discrete_map=color_map)  #asignar colores

#mostrar grafico
fig.show()


### Analisis 10:
* Objetivo: Identificar la cantidad de lineas telefonicas y su tendencia
* Metodo: Crear un grafico de lineas graficando valores historicos y creando una tendencia futura

In [37]:
#leer csv
df_tel_fij = pd.read_csv('datasets/telefonia_fija_Fija_accesos_tot.csv')

#borrar columnas que no se usaran
df_tel_fij.drop(columns=['periodo'], inplace=True)

#reorganizar el dataframe para el grafico
df_melted = df_tel_fij.melt(id_vars=['año', 'trimestre'], 
                              value_vars=[
                                  'accesos_telefonía_fija_(total)', 
                                  'accesos_telefonía_fija_(hogares)', 
                                  'accesos_telefonía_fija_(comercial)', 
                                  'accesos_telefonía_fija_(gobierno)', 
                                  'accesos_telefonía_fija_(otros)'], 
                              var_name='tipo', 
                              value_name='accesos')

#crear una nueva columna que combine el ano y el trimestre para la visualizacion
df_melted['fecha'] = df_melted['año'].astype(str) + '-' + df_melted['trimestre'].astype(str)

#aplicar promedio movil para suavizar los datos
df_melted['accesos_suavizados'] = df_melted.groupby('tipo')['accesos'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())

#crear una lista para almacenar los nuevos datos de proyeccion
projections = []

#crear proyecciones para 2025 y 2026
for year in [2025, 2026]:
    for tipo in df_melted['tipo'].unique():
        #obtener el ultimo acceso registrado
        last_access = df_melted[(df_melted['tipo'] == tipo) & (df_melted['año'] == 2024)]['accesos_suavizados'].values[0]
        #calcular un incremento basado en el crecimiento promedio
        growth_rate = df_melted[df_melted['tipo'] == tipo]['accesos_suavizados'].pct_change().mean()
        projected_access = last_access * (1 + growth_rate)
        #anadir las proyecciones a la lista
        projections.append({'año': year, 'trimestre': '1', 'tipo': tipo, 'accesos': projected_access, 'accesos_suavizados': projected_access, 'fecha': f'{year}-1'})

#convertir la lista de proyecciones en un dataframe
df_projections = pd.DataFrame(projections)

#concatenar los datos originales con las proyecciones
df_melted = pd.concat([df_melted, df_projections], ignore_index=True)

#crear grafico incial
fig = px.line(df_melted, 
              x='fecha', 
              y='accesos_suavizados', 
              color='tipo',
              title='Proyección de Accesos a la Telefonía Fija por Tipo',
              labels={'accesos_suavizados': 'Total de Accesos', 'fecha': 'Fecha'},
              markers=True)

#mostrar grafico
fig.show()


### Analisis 11:
* Objetivo: Identificar el tamaño de los tipos de lineas telefonicas
* Metodo: Crear un grafico circular representando cantidad de abonos

In [38]:
#leer csv
df_tel_fij = pd.read_csv('datasets/telefonia_fija_Fija_accesos_tot.csv')

#elegir primera fila o ultimo registro
first_row = df_tel_fij.iloc[0]

#crear el df
data = {
    'Tipo': ['Hogares', 'Comercial', 'Gobierno', 'Otros'],
    'Accesos': [
        first_row['accesos_telefonía_fija_(hogares)'], 
        first_row['accesos_telefonía_fija_(comercial)'], 
        first_row['accesos_telefonía_fija_(gobierno)'], 
        first_row['accesos_telefonía_fija_(otros)']
    ]
}
df_pie = pd.DataFrame(data)

#crear grafico inicial
fig = px.pie(df_pie, values='Accesos', names='Tipo', title='Tipos de Telefonía Fija',
             color_discrete_sequence=px.colors.qualitative.Set3)

#mostrar grafico inicial
fig.show()


### Analisis 12: KPI PROPUESTO 1
* Objetivo: Identificar el crecimiento de banda ancha fija vs portabilidad
* Metodo: Crear un grafico de lineas graficando una linea para cada tipo y prediciendo datos con linea de tendencia hasta 2026

In [45]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression

#leer archivos csv
df_tecnologia = pd.read_csv('datasets/Internet_Accesos_Por_Tecnologia.csv')
df_portabilidad = pd.read_csv('datasets/Portabilidad_Portin.csv')

#filtrar datos para eliminar 2024
df_tecnologia = df_tecnologia[df_tecnologia['año'] < 2024]
df_portabilidad = df_portabilidad[df_portabilidad['año'] < 2024]

#agrupar y sumar para obtener el total por ano para banda ancha fija
df_fija = df_tecnologia.groupby('año')['total'].sum().reset_index()
df_fija = df_fija.rename(columns={'total': 'total_fija'})

#agrupar y sumar para obtener el total de portabilidad por ano
df_portabilidad['total_portabilidad'] = df_portabilidad[['personal', 'claro', 'nextel', 'movistar']].sum(axis=1)
df_portabilidad = df_portabilidad.groupby('año')['total_portabilidad'].sum().reset_index()

#fusionar ambos dataframes
df_combined = pd.merge(df_fija, df_portabilidad, on='año', how='outer').sort_values(by='año')

#calcular el crecimiento promedio anual para ambos conjuntos de datos
growth_fija = df_combined['total_fija'].pct_change().mean()
growth_portabilidad = df_combined['total_portabilidad'].pct_change().mean()

#generar datos futuros (2024-2026) usando el crecimiento promedio
years_future = np.arange(2024, 2027)
future_fija = [df_combined['total_fija'].iloc[-1] * (1 + growth_fija) ** i for i in range(1, len(years_future) + 1)]
future_portabilidad = [df_combined['total_portabilidad'].iloc[-1] * (1 + growth_portabilidad) ** i for i in range(1, len(years_future) + 1)]

#agregar las proyecciones al dataframe original
df_future = pd.DataFrame({
    'año': years_future,
    'total_fija': future_fija,
    'total_portabilidad': future_portabilidad
})

df_combined = pd.concat([df_combined, df_future])

#crear el grafico de lineas con las dos series
fig = go.Figure()

#linea de acceso a banda ancha fija
fig.add_trace(go.Scatter(
    x=df_combined['año'], 
    y=df_combined['total_fija'], 
    mode='lines+markers', 
    name='Acceso Banda Ancha Fija',
    line=dict(color='blue')
))

#linea de portabilidad
fig.add_trace(go.Scatter(
    x=df_combined['año'], 
    y=df_combined['total_portabilidad'], 
    mode='lines+markers', 
    name='Portabilidad',
    line=dict(color='green')
))

#personalizar el grafico
fig.update_layout(
    title='Comparación de Banda Ancha Fija vs Portabilidad (Proyección hasta 2026)',
    xaxis_title='Año',
    yaxis_title='Total Accesos',
    hovermode='x',
    template='plotly_white'
)

#mostrar el grafico
fig.show()


### Analisis 13: KPI PROPUESTO 2
* Objetivo: Identificar el crecimiento de FTTH
* Metodo: Crear un grafico de lineas graficando una linea para cada tipo y prediciendo datos con linea de tendencia hasta 2026

In [48]:
import pandas as pd
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression
import numpy as np

# Leer el archivo CSV
df_tecnologia = pd.read_csv('datasets/Internet_Accesos_Por_Tecnologia.csv')

# Crear un diccionario que asocie las provincias con sus regiones
provincias_regiones = {
    'Buenos Aires': 'Pampeana', 'Capital Federal': 'Pampeana', 'Catamarca': 'Noroeste',
    'Chaco': 'Noreste', 'Chubut': 'Patagonia', 'Córdoba': 'Pampeana', 'Corrientes': 'Noreste',
    'Entre Ríos': 'Pampeana', 'Formosa': 'Noreste', 'Jujuy': 'Noroeste', 'La Pampa': 'Patagonia',
    'La Rioja': 'Noroeste', 'Mendoza': 'Cuyo', 'Misiones': 'Noreste', 'Neuquén': 'Patagonia',
    'Río Negro': 'Patagonia', 'Salta': 'Noroeste', 'San Juan': 'Cuyo', 'San Luis': 'Cuyo',
    'Santa Cruz': 'Patagonia', 'Santa Fe': 'Pampeana', 'Santiago del Estero': 'Noroeste',
    'Tierra del Fuego': 'Patagonia', 'Tucumán': 'Noroeste'
}

# Agregar la columna 'region' al DataFrame utilizando el diccionario
df_tecnologia['region'] = df_tecnologia['provincia'].map(provincias_regiones)

# Filtrar las columnas necesarias y eliminar filas con el año 2024
df_fibra = df_tecnologia[['año', 'region', 'fibra_óptica']]
df_fibra = df_fibra[df_fibra['año'] != 2024]

# Agrupar por año y región, sumando los accesos de fibra óptica
df_fibra_yearly = df_fibra.groupby(['año', 'region']).sum().reset_index()

# Crear proyecciones de crecimiento para 2024-2026 usando la línea de tendencia promedio
regiones = df_fibra_yearly['region'].unique()
años_proyeccion = [2024, 2025, 2026]
nuevos_datos = []

# Iterar sobre cada región para calcular el crecimiento promedio y proyectar datos futuros
for region in regiones:
    # Filtrar datos por región
    df_region = df_fibra_yearly[df_fibra_yearly['region'] == region]
    
    # Extraer los años y valores de accesos
    X = df_region['año'].values.reshape(-1, 1)
    y = df_region['fibra_óptica'].values

    # Ajustar un modelo de regresión lineal
    model = LinearRegression()
    model.fit(X, y)

    # Predecir los valores futuros para los años de proyección
    for año in años_proyeccion:
        prediccion = model.predict(np.array([[año]]))
        nuevos_datos.append({'año': año, 'region': region, 'fibra_óptica': prediccion[0]})

# Crear un DataFrame con los datos proyectados
df_proyeccion = pd.DataFrame(nuevos_datos)

# Concatenar los datos originales y los proyectados
df_fibra_total = pd.concat([df_fibra_yearly, df_proyeccion])

# Graficar la tendencia de fibra óptica con Plotly
fig = go.Figure()

# Agregar las líneas para cada región
for region in regiones:
    df_region = df_fibra_total[df_fibra_total['region'] == region]
    fig.add_trace(go.Scatter(
        x=df_region['año'],
        y=df_region['fibra_óptica'],
        mode='lines+markers',
        name=region,
        line_shape='spline'
    ))

# Configurar el diseño del gráfico
fig.update_layout(
    title='Crecimiento del Acceso a la Fibra Óptica por Región',
    xaxis_title='Año',
    yaxis_title='Accesos de Fibra Óptica',
    hovermode='x',
    template='plotly_white'
)

# Mostrar el gráfico
fig.show()
